<a href="https://colab.research.google.com/github/SahajSohi/Aircraft-Routing-Optimization-project/blob/main/Rough_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import random
import copy
import time
import plotly.express as px
import matplotlib.pyplot as plt
random.seed(0)
np.random.seed(0)
!pip install pyomo
!apt-get install -y glpk-utils

import pyomo.environ as pyo
from pyomo.opt import SolverFactory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 k

In [3]:
df=pd.read_excel('/content/Test_Case_1 (2).xlsx')

In [4]:
class airport:

    def __init__(self, aport, aircraftCount_dict):
        self.aport = aport
        self.aircraftCount_dict = aircraftCount_dict

class originAirport:

    def __init__(self, airport, deptime, depday):
        self.airport = airport
        self.deptime = deptime
        self.depday = depday
class destinationAirport:

    def __init__(self, airport, arrtime, arrday, elapsed_time, total_prop_delay, flight_time):
        self.airport = airport
        self.arrtime = arrtime
        self.arrday = arrday
        self.elapsed_time = elapsed_time
        self.total_prop_delay = total_prop_delay
        self.flight_time = flight_time

class FlightLeg:

    def __init__(self,origin,dest,flights,day,arr_delay,dep_datetime,arr_datetime,flighttime,
                 total_propagated_delay,pointer,elapsed_time, processed, firstnodeorigin, fleet):
        self.origin = origin
        self.dest = dest
        self.flights = flights
        self.day = day
        self.arr_delay = arr_delay
        self.dep_datetime = self.dateTimeObject(str(dep_datetime))
        self.arr_datetime = self.dateTimeObject(str(arr_datetime))
        self.flighttime = flighttime
        self.total_propagated_delay = total_propagated_delay
        self.pointer = pointer
        self.elapsed_time = elapsed_time
        self.processed = processed
        self.firstnodeorigin = firstnodeorigin
        self.fleet = fleet

    def dateTimeObject(self,String):
        return datetime.datetime.strptime(String,"%Y-%m-%d %H:%M:%S")

In [5]:
def calculate_slack_time_seconds(Dept,Arr, day1, day2, mean_turnaround_time):
    if day1 >= day2:
        return ((Dept-Arr).total_seconds() - mean_turnaround_time)
    else:
        return -99

def check_slack_time_seconds_validity(slack_time):
    return (24.*3600)>=slack_time>=0 #Check for flights within stipulated time for connection not beyond that

def check_time_elapsed_validity_seconds(time_elapsed):
    return (0<=time_elapsed<=36*60*60)

def calculate_propagated_delay_seconds(prev_prop_delay,arrival_delay,slack_time):
    prop_delay = prev_prop_delay+arrival_delay-slack_time # when arrival delay is in seconds
    return max(0, prop_delay)

def calculate_time_elapsed(prev_elapsed_time,Arr,Dept):
    return prev_elapsed_time+(Dept-Arr).total_seconds()

In [14]:
#Optimization problem
def route_optimizer(stagewise_routes, prop_delay, route_strings , assignedFlights, itr):

    model = ConcreteModel()
    n = len(stagewise_routes)

    uniqueFlights = []
    multipleFlight = []
    fp_dict = {}

    for sr in stagewise_routes:
        for s in sr:
            uniqueFlights.append(s)

    uniqueFlights = set(uniqueFlights)
